## Môi trường lập trình

In [3]:
# Thư viện
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import warnings
import joblib            as jlb
from sklearn.decomposition   import PCA
from sklearn.preprocessing   import LabelEncoder, OneHotEncoder
from sklearn.preprocessing   import Binarizer, PolynomialFeatures, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors       import KNeighborsClassifier
from sklearn                 import metrics
from sklearn.metrics         import accuracy_score, confusion_matrix
from sklearn.metrics         import auc,roc_curve
from sklearn.metrics         import precision_score, recall_score, f1_score
from sklearn.metrics         import roc_auc_score
from sklearn.model_selection import KFold
from sklearn                 import svm
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
from sklearn.model_selection import train_test_split

In [4]:
# Bỏ qua các cảnh báo
np.seterr(divide = 'ignore')
warnings.filterwarnings('ignore')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Tiền xử lý dữ liệu

In [5]:
df = pd.read_csv('Hotel Reservations.csv')

df.head(5)

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


### Thăm dò dữ liệu

Hiển thị thông tin bộ dữ liệu.

In [6]:
print(f'Kích thước của bộ dữ liệu gồm {df.shape[0]} dòng và {df.shape[1]} cột.')

Kích thước của bộ dữ liệu gồm 36275 dòng và 19 cột.


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36275 entries, 0 to 36274
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Booking_ID                            36275 non-null  object 
 1   no_of_adults                          36275 non-null  int64  
 2   no_of_children                        36275 non-null  int64  
 3   no_of_weekend_nights                  36275 non-null  int64  
 4   no_of_week_nights                     36275 non-null  int64  
 5   type_of_meal_plan                     36275 non-null  object 
 6   required_car_parking_space            36275 non-null  int64  
 7   room_type_reserved                    36275 non-null  object 
 8   lead_time                             36275 non-null  int64  
 9   arrival_year                          36275 non-null  int64  
 10  arrival_month                         36275 non-null  int64  
 11  arrival_date   

In [8]:
# In số giá trị duy nhất của mỗi cột
df.nunique()

Booking_ID                              36275
no_of_adults                                5
no_of_children                              6
no_of_weekend_nights                        8
no_of_week_nights                          18
type_of_meal_plan                           4
required_car_parking_space                  2
room_type_reserved                          7
lead_time                                 352
arrival_year                                2
arrival_month                              12
arrival_date                               31
market_segment_type                         5
repeated_guest                              2
no_of_previous_cancellations                9
no_of_previous_bookings_not_canceled       59
avg_price_per_room                       3930
no_of_special_requests                      6
booking_status                              2
dtype: int64

In [9]:
# In các giá trị duy nhất của các cột kiểu object
for column in df.select_dtypes(include=['object']).columns:
    unique_values = df[column].unique()
    print(f'Cột {column}:', unique_values)

Cột Booking_ID: ['INN00001' 'INN00002' 'INN00003' ... 'INN36273' 'INN36274' 'INN36275']
Cột type_of_meal_plan: ['Meal Plan 1' 'Not Selected' 'Meal Plan 2' 'Meal Plan 3']
Cột room_type_reserved: ['Room_Type 1' 'Room_Type 4' 'Room_Type 2' 'Room_Type 6' 'Room_Type 5'
 'Room_Type 7' 'Room_Type 3']
Cột market_segment_type: ['Offline' 'Online' 'Corporate' 'Aviation' 'Complementary']
Cột booking_status: ['Not_Canceled' 'Canceled']


In [10]:
# In các giá trị duy nhất của các cột kiểu number
for column in df.select_dtypes(include=['number']).columns:
    unique_values = df[column].unique()
    print(f'Cột {column}:', unique_values)

Cột no_of_adults: [2 1 3 0 4]
Cột no_of_children: [ 0  2  1  3 10  9]
Cột no_of_weekend_nights: [1 2 0 4 3 6 5 7]
Cột no_of_week_nights: [ 2  3  1  4  5  0 10  6 11  7 15  9 13  8 14 12 17 16]
Cột required_car_parking_space: [0 1]
Cột lead_time: [224   5   1 211  48 346  34  83 121  44   0  35  30  95  47 256  99  12
 122   2  37 130  60  56   3 107  72  23 289 247 186  64  96  41  55 146
  32  57   7 124 169   6  51  13 100 139 117  39  86  19 192 179  26  74
 143 177  18 267 155  46 128  20  40 196 188  17 110  68  73  92 171 134
 320 118 189  16  24   8  10 182 116 123 105 443 317 286 148  14  85  25
  28  80  11 162  82  27 245 266 112  88  69 273   4  97  31  62 197 280
 185 160 104  22 292 109 126 303  81  54  15 161 147  87 127 418 156  58
 433 111 195 119  59  78 335 103  70  76 144  49  77  36  79  21  33 164
 152  43 102  71 209  93  53 302 239  45 167 113  84   9 166 174  61 151
  52  67 282  38 175  89 133  65  66  50 159 386 115 237 125  91  29 221
 213 198  75 180 236 120

In [11]:
# Loại bỏ cột Booking_ID
df = df.drop(columns = ['Booking_ID'])

In [12]:
# Chuyển kiểu cột dữ liệu
df['repeated_guest'] = df['repeated_guest'].astype(object)
df['required_car_parking_space'] = df['required_car_parking_space'].astype(object)

In [13]:
# Đặt cột class là target
target = df['booking_status']
df = df.drop(columns = 'booking_status')

In [14]:
# In số giá trị duy nhất của cột target
tar_unique_values = target.unique()
tar_unique_values

array(['Not_Canceled', 'Canceled'], dtype=object)

### Làm sạch dữ liệu

Kiểm tra giá trị thiếu.

In [15]:
df.isnull().sum()

no_of_adults                            0
no_of_children                          0
no_of_weekend_nights                    0
no_of_week_nights                       0
type_of_meal_plan                       0
required_car_parking_space              0
room_type_reserved                      0
lead_time                               0
arrival_year                            0
arrival_month                           0
arrival_date                            0
market_segment_type                     0
repeated_guest                          0
no_of_previous_cancellations            0
no_of_previous_bookings_not_canceled    0
avg_price_per_room                      0
no_of_special_requests                  0
dtype: int64

Kiểm tra giá trị nhiễu.

In [16]:
# Sử dụng hàm để kiểm tra outlier cho tất cả các cột trong DataFrame
def check_outliers_using_3_sigma(df):
    has_outliers = True
    while has_outliers:
        has_outliers = False
        for col in df.columns:
            if df[col].dtype in ['int64', 'float64']:
                lech3sigma_tren = round(df[col].mean() + 3 * df[col].std(ddof=1))
                lech3sigma_duoi = round(df[col].mean() - 3 * df[col].std(ddof=1))
                df_filter = df[(df[col] > lech3sigma_tren) | (df[col] < lech3sigma_duoi)]

                if df_filter[col].count() > 0:
                    print(f'Cột {col} có {df_filter[col].count()} giá trị nhiễu là các giá trị ngoài khoảng ({lech3sigma_duoi}, {lech3sigma_tren})')
                    # Thay thế nhiễu bằng trung vị
                    trungvi = df[col].median()
                    df.loc[(df[col] > lech3sigma_tren) | (df[col] < lech3sigma_duoi), col] = trungvi
                    has_outliers = True
                else:
                    print(f'Cột {col}: Không có outlier.')

# Gọi hàm trên DataFrame df
check_outliers_using_3_sigma(df)

Cột no_of_adults có 16 giá trị nhiễu là các giá trị ngoài khoảng (0, 3)
Cột no_of_children có 1080 giá trị nhiễu là các giá trị ngoài khoảng (-1, 1)
Cột no_of_weekend_nights có 184 giá trị nhiễu là các giá trị ngoài khoảng (-2, 3)
Cột no_of_week_nights có 324 giá trị nhiễu là các giá trị ngoài khoảng (-2, 6)
Cột lead_time có 376 giá trị nhiễu là các giá trị ngoài khoảng (-173, 343)
Cột arrival_year: Không có outlier.
Cột arrival_month: Không có outlier.
Cột arrival_date: Không có outlier.
Cột no_of_previous_cancellations có 140 giá trị nhiễu là các giá trị ngoài khoảng (-1, 1)
Cột no_of_previous_bookings_not_canceled có 267 giá trị nhiễu là các giá trị ngoài khoảng (-5, 5)
Cột avg_price_per_room có 353 giá trị nhiễu là các giá trị ngoài khoảng (-2, 209)
Cột no_of_special_requests có 86 giá trị nhiễu là các giá trị ngoài khoảng (-2, 3)
Cột no_of_adults: Không có outlier.
Cột no_of_children: Không có outlier.
Cột no_of_weekend_nights: Không có outlier.
Cột no_of_week_nights: Không có out

Rời rạc hóa dữ liệu là các biến liên tục.

In [17]:
# Rời rạc các biến lead_time và avg_price_per_room
df['lead_time_group'] = pd.qcut(df['lead_time'], 4)
df['avg_price_per_room_group'] = pd.qcut(df['avg_price_per_room'], 4)
df = df.drop(columns = ['lead_time', 'avg_price_per_room'])

In [18]:
df = df.astype(object)

In [19]:
columns_to_exclude = ['type_of_meal_plan', 'room_type_reserved', 'market_segment_type']

# Chuyển giá trị ở các ô thành chuỗi có dạng "tên cột (giá trị ở ô tương ứng)" cho các cột trừ các cột có giá trị số rời rạc và nhị phân
df = df.apply(lambda x: x.astype(str).map(lambda y: f'{x.name} ({y})') if x.name not in columns_to_exclude else x)

In [20]:
df.shape

(36275, 17)

In [21]:
df.head(5)

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,no_of_special_requests,lead_time_group,avg_price_per_room_group
0,no_of_adults (2),no_of_children (0),no_of_weekend_nights (1),no_of_week_nights (2),Meal Plan 1,required_car_parking_space (0),Room_Type 1,arrival_year (2017),arrival_month (10),arrival_date (2),Offline,repeated_guest (0),no_of_previous_cancellations (0),no_of_previous_bookings_not_canceled (0),no_of_special_requests (0),"lead_time_group ((118.0, 309.0])","avg_price_per_room_group ((22.999, 81.08])"
1,no_of_adults (2),no_of_children (0),no_of_weekend_nights (2),no_of_week_nights (3),Not Selected,required_car_parking_space (0),Room_Type 1,arrival_year (2018),arrival_month (11),arrival_date (6),Online,repeated_guest (0),no_of_previous_cancellations (0),no_of_previous_bookings_not_canceled (0),no_of_special_requests (1),"lead_time_group ((-0.001, 17.0])","avg_price_per_room_group ((99.45, 118.96])"
2,no_of_adults (1),no_of_children (0),no_of_weekend_nights (2),no_of_week_nights (1),Meal Plan 1,required_car_parking_space (0),Room_Type 1,arrival_year (2018),arrival_month (2),arrival_date (28),Online,repeated_guest (0),no_of_previous_cancellations (0),no_of_previous_bookings_not_canceled (0),no_of_special_requests (0),"lead_time_group ((-0.001, 17.0])","avg_price_per_room_group ((22.999, 81.08])"
3,no_of_adults (2),no_of_children (0),no_of_weekend_nights (0),no_of_week_nights (2),Meal Plan 1,required_car_parking_space (0),Room_Type 1,arrival_year (2018),arrival_month (5),arrival_date (20),Online,repeated_guest (0),no_of_previous_cancellations (0),no_of_previous_bookings_not_canceled (0),no_of_special_requests (0),"lead_time_group ((118.0, 309.0])","avg_price_per_room_group ((99.45, 118.96])"
4,no_of_adults (2),no_of_children (0),no_of_weekend_nights (1),no_of_week_nights (1),Not Selected,required_car_parking_space (0),Room_Type 1,arrival_year (2018),arrival_month (4),arrival_date (11),Online,repeated_guest (0),no_of_previous_cancellations (0),no_of_previous_bookings_not_canceled (0),no_of_special_requests (0),"lead_time_group ((17.0, 57.0])","avg_price_per_room_group ((81.08, 99.45])"


In [22]:
df.dtypes

no_of_adults                            object
no_of_children                          object
no_of_weekend_nights                    object
no_of_week_nights                       object
type_of_meal_plan                       object
required_car_parking_space              object
room_type_reserved                      object
arrival_year                            object
arrival_month                           object
arrival_date                            object
market_segment_type                     object
repeated_guest                          object
no_of_previous_cancellations            object
no_of_previous_bookings_not_canceled    object
no_of_special_requests                  object
lead_time_group                         object
avg_price_per_room_group                object
dtype: object

### Chuẩn bị dữ liệu

In [23]:
def onehot(col, df):
  enc = OneHotEncoder(sparse = False) # dạng
  df_enc = pd.DataFrame(enc.fit_transform(df[[col]]), columns = df[col].unique())
  df.drop(columns = col, inplace = True)
  df_combined = pd.concat([df, df_enc], axis=1)
  return df_combined

In [24]:
for i in df.select_dtypes(include = ['object']).columns:
  df = onehot(i, df)

df = df.replace({1: True, 0: False})


# Kiểm thử giá trị minSup trong ngưỡng [0.1-0.8]

## minsup = 0.3


In [25]:
frequent_patterns = fpgrowth(df, min_support = 0.3, use_colnames=True)
frequent_patterns['itemsets'] = frequent_patterns['itemsets'].apply(lambda x: ', '.join(x))

# Sắp xếp chỉ số dựa trên số lượng từ
word_counts = frequent_patterns['itemsets'].str.split(",").apply(len)
sorted_indices = np.argsort(word_counts)

# Sắp xếp lại cột "Itemsets" dựa trên chỉ số đã sắp xếp
frequent_patterns['itemsets'] = frequent_patterns['itemsets'].iloc[sorted_indices].values

# Lưu các tập phổ biến lớn nhất
frequent_patterns_filter = frequent_patterns.copy()
frequent_patterns_reverse = frequent_patterns.iloc[::-1]
for valuehigh in frequent_patterns_reverse['itemsets']:
  for valuelow in frequent_patterns['itemsets']:
    dem = 0
    if len(valuelow.split(', ')) == len(valuehigh.split(', ')):
      break
    for l in valuelow.split(', '):
      if l in valuehigh.split(', '):
        dem = dem + 1
    if dem == len(valuelow.split(', ')):
      frequent_patterns_filter = frequent_patterns_filter[frequent_patterns['itemsets'] != valuelow]

In [26]:
# Lấy các cột là mẫu phổ biến
df_filtered = df[df.columns[df.columns.isin(frequent_patterns.itemsets)]]

## XOR của các tập phổ biến
for item in frequent_patterns_filter['itemsets']:
  row = []
  value = []
  for index in item.split(', '):
    row.append(index)
  name = ' x '.join(row)

  for i in row:
    value.append(df_filtered[i])

  result = value[0]
  for i in range(1, len(value)):
    result = [x ^ y for x, y in zip(result, value[i])]

  df_filtered[name] = result

# Xây dựng đặc trưng
features = df.astype(int).values

In [27]:
train = pd.concat([df_filtered, target], axis=1)

In [28]:
y = train['booking_status']
X = train.drop(columns = 'booking_status')

In [29]:
## Chia tập dữ liệu thành training, test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1)

## Xây dựng mô hình
model = svm.SVC(kernel = 'linear')
model.fit(X_train, y_train)

## Kiểm thử mô hình
y_pred = model.predict(X_test)

## đánh giá qua Accuracy
accuracy3  = accuracy_score(y_test, y_pred)

In [30]:
print(f'Giá trị accuracy của minSup=0.3 là:{accuracy3}')

Giá trị accuracy của minSup=0.3 là:0.735354927636113


## minsup = 0.4


In [31]:
frequent_patterns = fpgrowth(df, min_support = 0.4, use_colnames=True)
frequent_patterns['itemsets'] = frequent_patterns['itemsets'].apply(lambda x: ', '.join(x))

# Sắp xếp chỉ số dựa trên số lượng từ
word_counts = frequent_patterns['itemsets'].str.split(",").apply(len)
sorted_indices = np.argsort(word_counts)

# Sắp xếp lại cột "Itemsets" dựa trên chỉ số đã sắp xếp
frequent_patterns['itemsets'] = frequent_patterns['itemsets'].iloc[sorted_indices].values

# Lưu các tập phổ biến lớn nhất
frequent_patterns_filter = frequent_patterns.copy()
frequent_patterns_reverse = frequent_patterns.iloc[::-1]
for valuehigh in frequent_patterns_reverse['itemsets']:
  for valuelow in frequent_patterns['itemsets']:
    dem = 0
    if len(valuelow.split(', ')) == len(valuehigh.split(', ')):
      break
    for l in valuelow.split(', '):
      if l in valuehigh.split(', '):
        dem = dem + 1
    if dem == len(valuelow.split(', ')):
      frequent_patterns_filter = frequent_patterns_filter[frequent_patterns['itemsets'] != valuelow]

In [32]:
# Lấy các cột là mẫu phổ biến
df_filtered = df[df.columns[df.columns.isin(frequent_patterns.itemsets)]]

## XOR của các tập phổ biến
for item in frequent_patterns_filter['itemsets']:
  row = []
  value = []
  for index in item.split(', '):
    row.append(index)
  name = ' x '.join(row)

  for i in row:
    value.append(df_filtered[i])

  result = value[0]
  for i in range(1, len(value)):
    result = [x ^ y for x, y in zip(result, value[i])]

  df_filtered[name] = result

# Xây dựng đặc trưng
features = df.astype(int).values

In [33]:
train = pd.concat([df_filtered, target], axis=1)

In [34]:
y = train['booking_status']
X = train.drop(columns = 'booking_status')

In [35]:
## Chia tập dữ liệu thành training, test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1)

## Xây dựng mô hình
model = svm.SVC(kernel = 'linear')
model.fit(X_train, y_train)

## Kiểm thử mô hình
y_pred = model.predict(X_test)

## đánh giá qua Accuracy
accuracy4  = accuracy_score(y_test, y_pred)

In [36]:
print(f'Giá trị accuracy của minSup=0.4 là:{accuracy4}')

Giá trị accuracy của minSup=0.4 là:0.7390764989662302


## minsup = 0.5

In [37]:
frequent_patterns = fpgrowth(df, min_support = 0.5, use_colnames=True)
frequent_patterns['itemsets'] = frequent_patterns['itemsets'].apply(lambda x: ', '.join(x))

# Sắp xếp chỉ số dựa trên số lượng từ
word_counts = frequent_patterns['itemsets'].str.split(",").apply(len)
sorted_indices = np.argsort(word_counts)

# Sắp xếp lại cột "Itemsets" dựa trên chỉ số đã sắp xếp
frequent_patterns['itemsets'] = frequent_patterns['itemsets'].iloc[sorted_indices].values

# Lưu các tập phổ biến lớn nhất
frequent_patterns_filter = frequent_patterns.copy()
frequent_patterns_reverse = frequent_patterns.iloc[::-1]
for valuehigh in frequent_patterns_reverse['itemsets']:
  for valuelow in frequent_patterns['itemsets']:
    dem = 0
    if len(valuelow.split(', ')) == len(valuehigh.split(', ')):
      break
    for l in valuelow.split(', '):
      if l in valuehigh.split(', '):
        dem = dem + 1
    if dem == len(valuelow.split(', ')):
      frequent_patterns_filter = frequent_patterns_filter[frequent_patterns['itemsets'] != valuelow]

In [38]:
# Lấy các cột là mẫu phổ biến
df_filtered = df[df.columns[df.columns.isin(frequent_patterns.itemsets)]]

## XOR của các tập phổ biến
for item in frequent_patterns_filter['itemsets']:
  row = []
  value = []
  for index in item.split(', '):
    row.append(index)
  name = ' x '.join(row)

  for i in row:
    value.append(df_filtered[i])

  result = value[0]
  for i in range(1, len(value)):
    result = [x ^ y for x, y in zip(result, value[i])]

  df_filtered[name] = result

# Xây dựng đặc trưng
features = df.astype(int).values

In [39]:
train = pd.concat([df_filtered, target], axis=1)

In [40]:
y = train['booking_status']
X = train.drop(columns = 'booking_status')

In [41]:
## Chia tập dữ liệu thành training, test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1)

## Xây dựng mô hình
model = svm.SVC(kernel = 'linear')
model.fit(X_train, y_train)

## Kiểm thử mô hình
y_pred = model.predict(X_test)

## đánh giá qua Accuracy
accuracy5  = accuracy_score(y_test, y_pred)

In [42]:
print(f'Giá trị accuracy của minSup=0.5 là:{accuracy5}')

Giá trị accuracy của minSup=0.5 là:0.7389386629910406


## minsup = 0.6


In [43]:
frequent_patterns = fpgrowth(df, min_support = 0.6, use_colnames=True)
frequent_patterns['itemsets'] = frequent_patterns['itemsets'].apply(lambda x: ', '.join(x))

# Sắp xếp chỉ số dựa trên số lượng từ
word_counts = frequent_patterns['itemsets'].str.split(",").apply(len)
sorted_indices = np.argsort(word_counts)

# Sắp xếp lại cột "Itemsets" dựa trên chỉ số đã sắp xếp
frequent_patterns['itemsets'] = frequent_patterns['itemsets'].iloc[sorted_indices].values

# Lưu các tập phổ biến lớn nhất
frequent_patterns_filter = frequent_patterns.copy()
frequent_patterns_reverse = frequent_patterns.iloc[::-1]
for valuehigh in frequent_patterns_reverse['itemsets']:
  for valuelow in frequent_patterns['itemsets']:
    dem = 0
    if len(valuelow.split(', ')) == len(valuehigh.split(', ')):
      break
    for l in valuelow.split(', '):
      if l in valuehigh.split(', '):
        dem = dem + 1
    if dem == len(valuelow.split(', ')):
      frequent_patterns_filter = frequent_patterns_filter[frequent_patterns['itemsets'] != valuelow]

In [44]:
# Lấy các cột là mẫu phổ biến
df_filtered = df[df.columns[df.columns.isin(frequent_patterns.itemsets)]]

## XOR của các tập phổ biến
for item in frequent_patterns_filter['itemsets']:
  row = []
  value = []
  for index in item.split(', '):
    row.append(index)
  name = ' x '.join(row)

  for i in row:
    value.append(df_filtered[i])

  result = value[0]
  for i in range(1, len(value)):
    result = [x ^ y for x, y in zip(result, value[i])]

  df_filtered[name] = result

# Xây dựng đặc trưng
features = df.astype(int).values

In [45]:
train = pd.concat([df_filtered, target], axis=1)

In [46]:
y = train['booking_status']
X = train.drop(columns = 'booking_status')

In [47]:
## Chia tập dữ liệu thành training, test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1)

## Xây dựng mô hình
model = svm.SVC(kernel = 'linear')
model.fit(X_train, y_train)

## Kiểm thử mô hình
y_pred = model.predict(X_test)

## đánh giá qua Accuracy
accuracy6  = accuracy_score(y_test, y_pred)

In [48]:
print(f'Giá trị accuracy của minSup=0.6 là:{accuracy6}')

Giá trị accuracy của minSup=0.6 là:0.6763611302549966


## minsup = 0.7


In [49]:
frequent_patterns = fpgrowth(df, min_support = 0.7, use_colnames=True)
frequent_patterns['itemsets'] = frequent_patterns['itemsets'].apply(lambda x: ', '.join(x))

# Sắp xếp chỉ số dựa trên số lượng từ
word_counts = frequent_patterns['itemsets'].str.split(",").apply(len)
sorted_indices = np.argsort(word_counts)

# Sắp xếp lại cột "Itemsets" dựa trên chỉ số đã sắp xếp
frequent_patterns['itemsets'] = frequent_patterns['itemsets'].iloc[sorted_indices].values

# Lưu các tập phổ biến lớn nhất
frequent_patterns_filter = frequent_patterns.copy()
frequent_patterns_reverse = frequent_patterns.iloc[::-1]
for valuehigh in frequent_patterns_reverse['itemsets']:
  for valuelow in frequent_patterns['itemsets']:
    dem = 0
    if len(valuelow.split(', ')) == len(valuehigh.split(', ')):
      break
    for l in valuelow.split(', '):
      if l in valuehigh.split(', '):
        dem = dem + 1
    if dem == len(valuelow.split(', ')):
      frequent_patterns_filter = frequent_patterns_filter[frequent_patterns['itemsets'] != valuelow]

In [50]:
# Lấy các cột là mẫu phổ biến
df_filtered = df[df.columns[df.columns.isin(frequent_patterns.itemsets)]]

## XOR của các tập phổ biến
for item in frequent_patterns_filter['itemsets']:
  row = []
  value = []
  for index in item.split(', '):
    row.append(index)
  name = ' x '.join(row)

  for i in row:
    value.append(df_filtered[i])

  result = value[0]
  for i in range(1, len(value)):
    result = [x ^ y for x, y in zip(result, value[i])]

  df_filtered[name] = result

# Xây dựng đặc trưng
features = df.astype(int).values

In [51]:
train = pd.concat([df_filtered, target], axis=1)

In [52]:
y = train['booking_status']
X = train.drop(columns = 'booking_status')

In [53]:
## Chia tập dữ liệu thành training, test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1)

## Xây dựng mô hình
model = svm.SVC(kernel = 'linear')
model.fit(X_train, y_train)

## Kiểm thử mô hình
y_pred = model.predict(X_test)

## đánh giá qua Accuracy
accuracy7  = accuracy_score(y_test, y_pred)

In [54]:
print(f'Giá trị accuracy của minSup=0.7 là:{accuracy7}')

Giá trị accuracy của minSup=0.7 là:0.6763611302549966


## minsup = 0.8


In [57]:
frequent_patterns = fpgrowth(df, min_support = 0.8, use_colnames=True)
frequent_patterns['itemsets'] = frequent_patterns['itemsets'].apply(lambda x: ', '.join(x))

# Sắp xếp chỉ số dựa trên số lượng từ
word_counts = frequent_patterns['itemsets'].str.split(",").apply(len)
sorted_indices = np.argsort(word_counts)

# Sắp xếp lại cột "Itemsets" dựa trên chỉ số đã sắp xếp
frequent_patterns['itemsets'] = frequent_patterns['itemsets'].iloc[sorted_indices].values

# Lưu các tập phổ biến lớn nhất
frequent_patterns_filter = frequent_patterns.copy()
frequent_patterns_reverse = frequent_patterns.iloc[::-1]
for valuehigh in frequent_patterns_reverse['itemsets']:
  for valuelow in frequent_patterns['itemsets']:
    dem = 0
    if len(valuelow.split(', ')) == len(valuehigh.split(', ')):
      break
    for l in valuelow.split(', '):
      if l in valuehigh.split(', '):
        dem = dem + 1
    if dem == len(valuelow.split(', ')):
      frequent_patterns_filter = frequent_patterns_filter[frequent_patterns['itemsets'] != valuelow]

In [58]:
# Lấy các cột là mẫu phổ biến
df_filtered = df[df.columns[df.columns.isin(frequent_patterns.itemsets)]]

## XOR của các tập phổ biến
for item in frequent_patterns_filter['itemsets']:
  row = []
  value = []
  for index in item.split(', '):
    row.append(index)
  name = ' x '.join(row)

  for i in row:
    value.append(df_filtered[i])

  result = value[0]
  for i in range(1, len(value)):
    result = [x ^ y for x, y in zip(result, value[i])]

  df_filtered[name] = result

# Xây dựng đặc trưng
features = df.astype(int).values

In [59]:
train = pd.concat([df_filtered, target], axis=1)

In [60]:
y = train['booking_status']
X = train.drop(columns = 'booking_status')

In [61]:
## Chia tập dữ liệu thành training, test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1)

## Xây dựng mô hình
model = svm.SVC(kernel = 'linear')
model.fit(X_train, y_train)

## Kiểm thử mô hình
y_pred = model.predict(X_test)

## đánh giá qua Accuracy
accuracy8  = accuracy_score(y_test, y_pred)

In [62]:
print(f'Giá trị accuracy của minSup=0.8 là:{accuracy8}')

Giá trị accuracy của minSup=0.8 là:0.6763611302549966


## Tổng hợp

In [63]:
print(f'Giá trị accuracy của minSup=0.3 là:{accuracy3}')
print(f'Giá trị accuracy của minSup=0.4 là:{accuracy4}')
print(f'Giá trị accuracy của minSup=0.5 là:{accuracy5}')
print(f'Giá trị accuracy của minSup=0.6 là:{accuracy6}')
print(f'Giá trị accuracy của minSup=0.7 là:{accuracy7}')
print(f'Giá trị accuracy của minSup=0.8 là:{accuracy8}')

Giá trị accuracy của minSup=0.3 là:0.735354927636113
Giá trị accuracy của minSup=0.4 là:0.7390764989662302
Giá trị accuracy của minSup=0.5 là:0.7389386629910406
Giá trị accuracy của minSup=0.6 là:0.6763611302549966
Giá trị accuracy của minSup=0.7 là:0.6763611302549966
Giá trị accuracy của minSup=0.8 là:0.6763611302549966
